# Demand Letter Analsysis using Swarm of Strands Agents 

## Understanding Multi-Agent Systems and Swarm Intelligence

An agent swarm is a collection of autonomous AI agents working together to solve complex problems through collaboration. Inspired by natural systems like ant colonies or bird flocks, agent swarms leverage collective intelligence where the combined output exceeds what any single agent could produce. By distributing tasks and sharing information, swarms can tackle complex problems more efficiently and effectively than individual agents working in isolation.

Multi-agent systems consist of multiple interacting intelligent agents within an environment. These systems enable:

- *Distributed Problem Solving*: Breaking complex tasks into subtasks for parallel processing
- *Information Sharing*: Agents exchange insights to build collective knowledge
- *Specialization*: Different agents focus on specific aspects of a problem
- *Redundancy*: Multiple agents working on similar tasks improve reliability
- *Emergent Intelligence*: The system exhibits capabilities beyond those of its individual components

Swarm intelligence emphasizes:

1. *Decentralized Control*: No single agent directs the entire system
2. *Local Interactions*: Agents primarily interact with nearby agents
3. *Simple Rules*: Individual agents follow relatively simple behaviors
4. *Emergent Complexity*: Complex system behavior emerges from simple agent interactions

## 1. Quick Start with Swarm tool

The Strands Agents SDK provides a built-in swarm tool that simplifies the implementation of multi-agent systems, offering a quick start for users. This tool implements the shared memory.

In [1]:
!pip install -r requirements.txt
!pip install PyPDF2

In [2]:
from strands import Agent
from strands_tools import swarm
import time
import PyPDF2
import json
import boto3
import yaml
import os
import logging


In [3]:
import PyPDF2

def read_pdf(file_path):
    """
    Read and extract text from a PDF file.
    
    Args:
        file_path (str): Path to the PDF file
        
    Returns:
        str: Extracted text from the PDF
    """
    try:
        # Open the PDF file in binary read mode
        with open(file_path, 'rb') as file:
            # Create a PDF reader object
            pdf_reader = PyPDF2.PdfReader(file)
            
            # Get number of pages
            num_pages = len(pdf_reader.pages)
            print(f"Total pages: {num_pages}")
            
            # Extract text from each page
            text = ""
            for page_num in range(num_pages):
                page = pdf_reader.pages[page_num]
                text += page.extract_text()
                
            return text
    
    except FileNotFoundError:
        return "Error: The file was not found."
    except PyPDF2.errors.PdfReadError:
        return "Error: Invalid PDF file or the file is encrypted."
    except Exception as e:
        return f"Error: {str(e)}"



In [4]:
# Example usage
if __name__ == "__main__":
    pdf_path = "data/LEGALCORRESPONDENCE.pdf"  # Replace with your PDF file path
    extracted_text = read_pdf(pdf_path)
    print("Extracted text:")
    print(extracted_text[0:1000])

Total pages: 14
Extracted text:
LEGAL CORRESPONDENCE: PRIVILEGED & CONFIDENTIAL CTRL, ALT & DELETE, LLP A3orneys at Law 404 Error Lane,  Suite 400 United States of Algorithm, D.C. 101010 Tel: (234) 713-1719 Fax: (123) 581-3213 www.randomlegal.com  May 15, 2023  VIA CERTIFIED MAIL (RETURN RECEIPT REQUESTED) AND ELECTRONIC MAIL  Mr. Richard Henderson Senior Claims Oﬃcer Octank Insurance Company, Inc. 800 Financial Place, 35th Floor New York, NY 011011 rhenderson@ocktankinsurance.com  Re: Insured: Westlake Commercial Proper_es, LLC Policy No.: PCCP-785493-22 Date of Loss: December 8, 2022 Claim No.: PIN-2022-78549 Loca_on: 478 Industrial Parkway, Sea3le, WA Our File No.: TWM-23-0472  DEMAND LETTER AND NOTICE OF BAD FAITH CLAIMS HANDLING PRACTICES  Dear Mr. Henderson:  Please be advised that this ﬁrm has been retained to represent the interests of Westlake Commercial Proper_es, LLC ("Westlake" or "Insured") with respect to the above-referenced ﬁrst-party commercial property insurance claim

### 1.1 Direct tool innvocation

In [5]:
agent = Agent(tools=[swarm])

In [6]:
prompt= f"""
             You are an Insurance Claim Legal Expert. You task is to review the {extracted_text} and do the following:
             1) Verify: Review the letter and verify this is a demand letter from a legal firm
             2) Classify: Carefully understand the language and Classify this document in Urgent-Response , or Normal-Response. Pay special
             attention to the amount of money or damages that is demanded and how soon (days, hours) they want a response.
                 Urgent-Response: Is the ammount of money in the demand letter exceeding $50,000 and/or the response time frame is less than 3 days (72 hours)
                 Normal-Response: The ammount of money in the demand letter is less than $50,000 and/or we have more than 3 days (72 hours) to respond
             3) Validate: Evalute the language and Policy provisions to validate if the damages, limits and provisions are in compliance with Insurance Policies
             4) Write an email that corresponds to the Classification: Urgent-Response or Normal-Response. If you determine that the demand has no grounds, then include that in the email and tell them to "go to away" using legal language
             """

In [7]:
result = agent.tool.swarm(
    task=prompt,
    swarm_size=3,
    coordination_pattern="collaborative",
)

# Review of Demand Letter from CTRL, ALT & DELETE, LLP

## 1) Verification
I confirm this is a demand letter from legal firm CTRL, ALT & DELETE, LLP representing Westlake Commercial Properties, LLC# Analysis of Westlake Commercial Properties Demand Letter

##  against Octank Insurance Company. The letter follows1) Verification
I confirm this is a formal demand letter from the# Document Analysis: Legal Demand Letter Review

##  law firm CTRL,1) Verification
This is clearly a formal demand letter from a ALT & DELETE legal firm calle, LLP representingd CTRL, AL WestlakeT & DELETE, Commercial Properties, LLP, representing formal legal demand format with letterhead, attorney LLC against Westlake Oct Commercial Properties, LLCank Insurance Company regarding against Ockt claim signature, claim references, and specificank Insurance Company. PIN The letter follows- standard legal formatting,2 allegations of bad faith insurance practices.

## includes022 appropriate-78 case549 references 2) Clas

Here you can see that the 3 agents built upon others'insights and seek consensus

In [8]:
result["content"][14:15]

[{'text': '\n🌟 Collective Knowledge:\n[\n  {\n    "id": "1922f554-c665-497e-84e1-079094755826",\n    "agent_id": "agent_1",\n    "content": "Response: # Document Analysis: Legal Demand Letter Review\\n\\n## 1) Verification\\nThis is clearly a formal demand letter from a legal firm called CTRL, ALT & DELETE, LLP, representing Westlake Commercial Properties, LLC against Ocktank Insurance Company. The letter follows standard legal formatting, includes appropriate case references, and is signed by Margaret E. Alt, Esq., Senior Partner in the Insurance Recovery Practice Group.\\n\\n## 2) Classification\\n**Classification: Urgent-Response**\\n\\nThe demand letter meets both urgent criteria:\\n- Amount demanded: ONE MILLION DOLLARS ($1,000,000.00)\\n- Response timeframe: 72 hours (with explicit statement that failure to respond within this timeframe \\"will be deemed a waiver of all policy limits\\")\\n\\nThe letter also includes threats of bad faith litigation, requests for a litigation hold

In [9]:
summarizer_agent = Agent(system_prompt="""You are a Summarizer Agent specializing in synthesizing information.
Your role in the swarm is to gather insights from all agents and create a cohesive final response.
You should combine the best ideas and address the criticisms to create a comprehensive response.
Focus on creating a clear, actionable summary that addresses the original query effectively.
""")

In [10]:
query = f"""
             Original query: {result["content"][2:]}
             Cosolidate the response from the swarm and provide a summary for:
             1)Vericy
             2)Classify
             3)Validate
             4)Email
             """



In [11]:
final_response_collaboration = str(summarizer_agent(query))



# Comprehensive Analysis of Westlake Commercial Properties Demand Letter

## 1) Verification
This is a formal demand letter from CTRL, ALT & DELETE, LLP representing Westlake Commercial Properties, LLC against Octank Insurance Company. The letter contains:
- Proper legal formatting with letterhead and attorney signature (Margaret E. Alt, Esq.)
- Specific claim references (Policy No. PCCP-785493-22, Claim No. PIN-2022-78549)
- Detailed allegations of bad faith insurance practices
- References to multiple expert reports and specific Washington regulations
- Formal monetary demands and response requirements

## 2) Classification
**Classification: URGENT-RESPONSE**

The letter meets urgent response criteria for multiple reasons:
- Monetary demand of $1,000,000.00 (significantly exceeding the $50,000 threshold)
- 72-hour response requirement (meeting the 3-day threshold)
- Explicit statement that failure to respond within timeframe "will be deemed a waiver of all policy limits"
- Includes a

In [12]:
final_response_collaboration

'# Comprehensive Analysis of Westlake Commercial Properties Demand Letter\n\n## 1) Verification\nThis is a formal demand letter from CTRL, ALT & DELETE, LLP representing Westlake Commercial Properties, LLC against Octank Insurance Company. The letter contains:\n- Proper legal formatting with letterhead and attorney signature (Margaret E. Alt, Esq.)\n- Specific claim references (Policy No. PCCP-785493-22, Claim No. PIN-2022-78549)\n- Detailed allegations of bad faith insurance practices\n- References to multiple expert reports and specific Washington regulations\n- Formal monetary demands and response requirements\n\n## 2) Classification\n**Classification: URGENT-RESPONSE**\n\nThe letter meets urgent response criteria for multiple reasons:\n- Monetary demand of $1,000,000.00 (significantly exceeding the $50,000 threshold)\n- 72-hour response requirement (meeting the 3-day threshold)\n- Explicit statement that failure to respond within timeframe "will be deemed a waiver of all policy lim

How about scenarios where you need Competitiveness?  

In [13]:
result = agent.tool.swarm(
    task=prompt,
    swarm_size=3,
    coordination_pattern="competitive",
)

# Demand Letter Analysis and Response

### Insurance Demand Letter Analysis

## 1. Verification
I confirm this is a formal 1) Verification demand letter from the legal firm CTRL, ALT &# Analysis of Westlake Commercial Properties, LLC Demand Letter

## 1) Verification
This is definitively a formal demand letter from DELETE, LLP the
I confirm this is a demand letter law representing firm CTRL, Westlake ALT & DELETE Commercial Properties, LLC, LLP, against representing Westlake Ocktank Commercial Properties, LLC Insurance Company. from the law firm CTRL,. The letter follows The letter follows standard legal standard legal ALT & DELETE, LLP. deman format withd letter format, includes proper letter properhead, letter referencehead, case reference information information, and is, and is signed by Margaret signe The letter bears all formal legal E. Alt,d by Margaret E. Alt, Esq., Esq., Senior Senior Partner of the firm Partner of the's firm's Insurance Recovery Practice Group Insurance Recove

In [14]:
summarizer_agent = Agent(system_prompt="""You are a Summarizer Agent specializing in synthesizing information.
Your role in the swarm is to gather insights from all agents and create a cohesive final response.
You should combine the best ideas and address the criticisms to create a comprehensive response.
Focus on creating a clear, actionable summary that addresses the original query effectively.
""")

In [15]:
query = f"""
             Original query: {result["content"][2:]}
             Cosolidate the response from the swarm and provide a summary for:
             1)Vericy
             2)Classify
             3)Validate
             4)Email
             """


In [16]:
final_response_competition = str(summarizer_agent(query))

# Comprehensive Demand Letter Analysis & Response

## 1) Verification
The document is conclusively a formal legal demand letter from CTRL, ALT & DELETE, LLP representing Westlake Commercial Properties, LLC against Ocktank Insurance Company. It contains all standard legal elements including letterhead, certified mail designation, attorney signature block, detailed case references, and explicit legal demands.

## 2) Classification: URGENT-RESPONSE
This matter requires immediate urgent response based on multiple critical factors:
- **Financial threshold**: $1,000,000 demand (significantly exceeding the $50,000 threshold)
- **Response timeline**: 72-hour deadline (exactly at the 3-day threshold)
- **Additional urgency factors**:
  * Warning that failure to respond constitutes "waiver of all policy limits" 
  * Allegations of statutory violations under Washington Insurance Fair Conduct Act
  * Notification to state regulatory authorities
  * Litigation hold notice indicating imminent legal 

In [17]:
result["content"][2:]

[{'text': '🤖 Agent agent_1: Response: # Demand Letter Analysis and Response\n\n## 1) Verification\nI confirm this is a demand letter from the law firm CTRL, ALT & DELETE, LLP. The letter bears all formal legal correspondence elements including letterhead, certified mail notification, attorney signature, case references, and legal citations. It contains specific demands, allegations of bad faith, and a litigation hold notice.\n\n## 2) Classification\n**Classification: Urgent-Response**\n\nJustification:\n- Demand amount: $1,000,000.00 (significantly exceeds $50,000 threshold)\n- Response timeframe: 72 hours (exactly 3 days)\n- Contains severe warnings that failure to respond within this timeframe "will be deemed a waiver of all policy limits"\n- Involves allegations of statutory violations and bad faith claims handling\n- Threatens immediate litigation if demands aren\'t met\n\n## 3) Validation\nAnalysis of policy provisions and damages claims:\n- The policy does appear to provide flood

The swarm implements different collaboration strategies through specialized agent roles:

- Collaborative Pattern: Agents build upon others' insights and seek consensus
- Competitive Pattern: Agents develop independent solutions and unique perspectives

### 1.2 Natural Language Invocation

In [18]:
# During initialization
agent = Agent(tools=[swarm])

# Invocation through natural language
result = str(agent(
    f"""Use a swarm of 3 agents to execute the tasks describe in: {prompt}"""
))

I'll use a swarm of 3 AI agents to analyze this legal demand letter and complete the requested tasks. Let me set this up for you.
Tool #1: swarm
# Insurance Claim Deman## Demand Letter Assessment Phase 0 -d Letter Review

I'll analyze Initial Analysis

I Contribution notice this demand letter from CTRL the demand letter hasn - Agent 3

After't been provided in, ALT & DELETE, LLP reviewing the task the task description, I notice. on behalf of Westlake Commercial To we nee properly complete Properties to Octank Insurance this task, I would to analyze a demand need to review the Company.

## Task 1: actual demand letter from CTRLd letter but haven, ALT & DELETE, LLP Legitimacy Verification't been provided with the actual on behalf of Westlake Commercial letter content Properties, LLC to Octank Insurance. To properly Company.

To
To properly verify legitimacy, I woul effectively analyze the complete the assigne demand letter, I would need to see the actual letter contentd tasks, I would ne

I notice that there was an issue with the swarm analysis. The agents didn't appear to have access to the demand letter content that was included in your request. Let me analyze the demand letter myself based on the tasks you've outlined.

## 1) Verify: Review the letter and verify this is a demand letter from a legal firm

Yes, this is clearly a demand letter from a legal firm. The letter is from CTRL, ALT & DELETE, LLP (a law firm) on behalf of Westlake Commercial Properties, LLC to Octank Insurance Company. It contains all the proper legal formatting including letterhead, contact information, reference numbers, and is signed by Margaret E. Alt, Esq., Senior Partner in the Insurance Recovery Practice Group.

## 2) Classify: Urgent-Response or Normal-Response

**Classification: URGENT-RESPONSE**

The letter:
- Demands payment of $1,000,000.00
- Requires a response within 72 hours (three days)
- Contains explicit legal threats including potential litigation for bad faith insurance pract

In [19]:
print(result)

I notice that there was an issue with the swarm analysis. The agents didn't appear to have access to the demand letter content that was included in your request. Let me analyze the demand letter myself based on the tasks you've outlined.

## 1) Verify: Review the letter and verify this is a demand letter from a legal firm

Yes, this is clearly a demand letter from a legal firm. The letter is from CTRL, ALT & DELETE, LLP (a law firm) on behalf of Westlake Commercial Properties, LLC to Octank Insurance Company. It contains all the proper legal formatting including letterhead, contact information, reference numbers, and is signed by Margaret E. Alt, Esq., Senior Partner in the Insurance Recovery Practice Group.

## 2) Classify: Urgent-Response or Normal-Response

**Classification: URGENT-RESPONSE**

The letter:
- Demands payment of $1,000,000.00
- Requires a response within 72 hours (three days)
- Contains explicit legal threats including potential litigation for bad faith insurance pract

The swarm tool implements a SharedMemory system that serves as a central knowledge repository for all agents in the swarm. This system maintains a thread-safe store where agents can record their contributions with metadata (including agent ID, content, phase, and timestamp). It tracks processing phases, allowing agents to retrieve only current-phase knowledge or access historical information. This shared memory architecture enables concurrent collaboration, maintains contribution history, and ensures smooth information flow between agents—all essential features for effective collective intelligence in a swarm.

The full implementation of the swarm tool can be found in the [Strands Tools repository](https://github.com/strands-agents/tools/blob/main/src/strands_tools/swarm.py).

Key Parameters
- task: The main task to be processed by the swarm
- swarm_size: Number of agents in the swarm (1-10)
- coordination_pattern: How agents should coordinate
  - collaborative: Agents build upon others' insights
  - competitive: Agents develop independent solutions
- hybrid: Balances cooperation with independent exploration

How the Swarm Tool Works
1. *Initialization*: Creates a swarm with shared memory and specialized agents
2. *Phase Processing*: Agents work in parallel using ThreadPoolExecutor
3. *Knowledge Sharing*: Agents store and retrieve information from shared memory
4. *Result Collection*: Results from all agents are aggregated and presented


## 2. Creating a Swarm with Strands Agents

Strands Agents SDK allows you to create swarms using existing Agent objects, even when they use different model providers or have different configurations. While various communication architectures are possible (hierarchical, parallel, sequential, and mesh), the following example demonstrates a mesh architecture implementation, which provides a flexible foundation for agent-to-agent communication.

### 2.1 Mesh Swarm Architecture

<p align="center">
    <img src="./images/swarm_example.png">
</p>

In a mesh architecture, all agents can communicate directly with each other. The following example demonstrates a swarm of specialized agents using mesh communication to solve problems collaboratively:

This mesh architecture enables direct communication between all agents, allowing each agent to share insights with every other agent. The specialized roles (research, creative, critical, and summarizer) work together to produce a comprehensive solution that benefits from multiple perspectives and iterative refinement.

### 2.2 Implementing Shared Memory

While the mesh communication example effectively demonstrates agent collaboration, a shared memory system would enhance the swarm's capabilities by providing:

- A centralized knowledge repository for all agents
- Automated phase tracking and historical knowledge preservation
- Thread-safe concurrent access for improved efficiency
- Persistent storage of insights across multiple interactions

Extending our mesh swarm example with shared memory would replace the message dictionary with a SharedMemory instance, simplifying the code while enabling more sophisticated knowledge management.

## When to use Swarm:

- For quick, parallel processing of a single complex task
- When you need multiple perspectives on the same problem
- For tasks that benefit from collective intelligence

## Conclusion

Multi-agent swarms solve complex problems through collective intelligence. The Strands Agents SDK supports both custom implementations and a built-in swarm tool with shared memory. By distributing tasks across specialized agents and enabling effective communication, swarms achieve better results than single agents working alone. Whether using mesh communication patterns or the swarm tool, developers can create systems where multiple agents work together with defined roles, coordination mechanisms, and knowledge sharing.

